In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('#100')

['#', '100']

In [4]:
def stem(word):
    return stemmer.stem(word)

In [5]:
stem('organic')

'organ'

In [ ]:
def bag_of_words(tokenzied_words,all_words):
    